In [1]:
import sys
sys.path.append('..')
import os
from os.path import expanduser

import lcatools

In [2]:
from lcatools.tools import archive_from_json

In [3]:
elcd3_file = os.path.join(os.path.expanduser('~'),'Dropbox', 'data', 'ELCD', 'ELCD3.2.zip')
catalog_dir = os.path.join(os.path.expanduser('~'), 'GitHub', 'lca-tools-datafiles', 'catalogs')
lcia_file = os.path.join(catalog_dir, 'elcd_lcia.json.gz')

In [4]:
l = archive_from_json(lcia_file)

Loading JSON data from /home/b/GitHub/lca-tools-datafiles/catalogs/elcd_lcia.json.gz:
Found Extension: zip
**Upstream reference encountered: /home/b/Dropbox/data/ELCD/ELCD3.2.zip

54 new quantity entities added (54 total)
1256 new flow entities added (1256 total)
0 new process entities added (0 total)


In [5]:
from lcatools.providers.ilcd import IlcdArchive

In [6]:
ps = IlcdArchive(elcd3_file, upstream=l)

Found Extension: zip


In [7]:
ps.retrieve_or_fetch_entity(ps.list_objects('Process')[27])

In [8]:
p = _

In [9]:
qs = (x for x in l.quantities() if 'Method' in x.keys())

In [10]:
q = next(qs)

In [11]:
q.keys()

dict_keys(['Comment', 'Method', 'Name', 'origin', 'Category', 'ReferenceYear', 'Duration', 'Indicator', 'externalId'])

In [12]:
def lcia(process, lcia):
    print(', '.join([lcia[desc] for desc in ('Method', 'Category', 'Indicator')]))
    print('-'*60)
    score = 0.0
    for x in process.exchanges():
        if x.flow.cf(lcia) != 0:
            c = x.flow.cf(lcia)
            v = x.value
            score += c*v
            print('%10.3g x %-10.3g = %-10.3g %s' % (c, v, c*v, x.flow['Name']) )
    print('='*60)
    print('             Total score: %g [%s]' % (score, lcia.reference_entity.unitstring()))
    return score

In [23]:
%time lcia(p, next(qs))

ILCD2011, Ionizing radiation, Comparative Toxic Unit for ecosystems (CTUe) expressing an estimate of the potentially affected fraction of species (PAF) integrated over time and volume per unit mass of a radionucleide emitted (PAF m3 year/kg).
------------------------------------------------------------
  1.86e-05 x 0.00089    = 1.65e-08   antimony-124
  1.51e-05 x 0.0273     = 4.12e-07   antimony-125
  1.04e-05 x 0.639      = 6.65e-06   iodine-131
  4.59e-09 x 133        = 6.13e-07   hydrogen-3
  2.43e-05 x 0.144      = 3.51e-06   cobalt-58
  4.62e-05 x 0.0242     = 1.12e-06   antimony-124
   5.5e-05 x 0.000267   = 1.47e-08   cesium-134
  0.000124 x 0.0275     = 3.41e-06   cesium-134
  2.26e-05 x 0.000968   = 2.19e-08   cobalt-60
  4.78e-06 x 9.97e-10   = 4.77e-15   manganese-54
  4.49e-05 x 0.000355   = 1.59e-08   cesium-137
  0.000172 x 0.0207     = 3.56e-06   carbon-14
  1.37e-05 x 0.0295     = 4.04e-07   silver-110
  6.83e-05 x 4.24e-06   = 2.9e-10    tellurium-123
  5.41e-05 x 0.1

0.0031144941582810586